<a href="https://colab.research.google.com/github/mohd-faizy/05P_Understanding_Deepfakes_with_Keras_Using_DCGAN/blob/master/Understanding_Deepfakes_with_Keras_Using_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Understanding Deepfakes with Keras Using DCGAN [Fix The code]__


## __What is DCGAN__

- __DCGAN__ stands for __Deep Convolutional Generative Adversarial Networks__.

- it was proposed by __Alec Radford et al. in late 2015__.






<center><img src='https://image.slidesharecdn.com/dlcvd4l1generativemodelsandaversarialtraining-160803172437/95/deep-learning-for-computer-vision-generative-models-and-adversarial-training-upc-2016-5-638.jpg?cb=1470245137'></center>


- The __Generator__ and __Discriminator__ as two black-box functions that map an input to an output. 

- The __Generator__ takes a vector of random numbers (either normally or uniformly distributed) and convert them to an image. 

- On the other hand, the __Discriminator__ has two input options: when it is fed with a real image, it should say "real!", and output a probabilistic number close to 1; when it is fed with a generated image, it should say "fake!", and output a number close to 0.

- By __competing with each other__, both the generator and discriminator will get better. Ultimately, _the generator is able to create images so real that the discriminator can no longer differentiate_, which marks the end of the game. This whole training mechanism is named __Generative Adversarial Networks (GAN)__, proposed by __Ian Goodfellow et al. in 2014.__


- __DCGAN__ is one of the popular and successful network design for __GAN__. It mainly composes of __convolution layers__ without __max pooling__ or __fully connected layers.__

- It uses __convolutional stride__ and transposed convolution for the __downsampling__ and the __upsampling__. 

_The figure below is the network design for the generator._

<center><img src='https://miro.medium.com/max/700/1*KvMnRfb76DponICrHIbSdg.png'></center>

__Here is the summary of DCGAN:__

- __Replace all max pooling with convolutional stride__
- __Use transposed convolution for upsampling.__
- __Eliminate fully connected layers.__
- __Use Batch normalization except the output layer for the generator and the input layer of the discriminator.__
- __Use ReLU in the generator except for the output which uses tanh.__
- __Use LeakyReLU in the discriminator.__

### __Oth_Resources__
- [__TenserFlow.org:__ Deep Convolutional Generative Adversarial Network(DCGANS)](https://www.tensorflow.org/tutorials/generative/dcgan)

- [__ArXive.org:__ Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434)

- [__gluon.mxnet.io__: DCGAN](https://gluon.mxnet.io/chapter14_generative-adversarial-networks/dcgan.html)

> The simplicity of DCGAN contributes to its success. We reach certain bottleneck that increasing the complexity of the generator does not necessarily improve the image quality. Until we identify the bottleneck and know how to train GANs more effective, DCGAN remains a good start point for a new project.

In [ ]:
!pip3 install git+https://github.com/mohd-faizy/tfutils

  Cloning https://github.com/mohd-faizy/tfutils to /tmp/pip-req-build-fpm8r59_
  Running command git clone -q https://github.com/mohd-faizy/tfutils /tmp/pip-req-build-fpm8r59_
  Created wheel for tfutils: filename=tfutils-0.0.1-cp36-none-any.whl size=6442 sha256=5a6f7aee7ba323df967aa96578d7b54d286a39d10841914b490777431b2f91c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-31fc6x0l/wheels/a5/50/97/f42b151f901199c149996d89c28d95c2749e557fbcbf48b9d2
Successfully built tfutils


# __1. Importing Libraries and Helper Functions__

$\color{red}{\textbf{Note}}$: If you haven't already, please install the required packages by executing the code cell below.

In [ ]:
%matplotlib notebook

import tensorflow as tf
import numpy as np
import os
import tfutils

from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose, Reshape, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from PIL import Image

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0


# __2. Importing and Plotting the Data__

$OneHotEncoding$

- [__What is One Hot Encoding? Why And When do you have to use it?__](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)


- [__Why One-Hot Encode Data in Machine Learning?__](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/)

- [__What is One Hot Encoding and How to Do It__](https://medium.com/@michaeldelsole/what-is-one-hot-encoding-and-how-to-do-it-f0ae272f1179)

- [__One-Hot Encoding vs. Label Encoding using Scikit-Learn__](https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/)

The function below downlode the __MNIST__ DataSet and Preprocess it to normalise.

-  The __MNIST database__ (_Modified National Institute of Standards and Technology database_) is a large database of __handwritten digits__ that is commonly used for training various image processing systems 

- Image has __pixel value__ from 0-255

In [ ]:
# Downloding the dataset
(x_train, y_train), (x_test, y_test) = tfutils.datasets.mnist.load_data(one_hot=False)

# Loading the Subsets that belong to the class zero 
# So the x_train, x_test have the images of only '0'
x_train = tfutils.datasets.mnist.load_subset([0], x_train, y_train)
x_test = tfutils.datasets.mnist.load_subset([0], x_test, y_test)

# Creating the Combined set Using the NumPy Concatenate function
x = np.concatenate([x_train, x_test], axis=0)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
tfutils.datasets.mnist.plot_ten_random_examples(plt, x, np.zeros((x.shape[0], 1))).show()

<IPython.core.display.Javascript object>

# __3. Discriminater__

<img src = 'https://media.geeksforgeeks.org/wp-content/uploads/gans_gfg.jpg'>

> [__Generative Adversarial Networks__](https://arxiv.org/abs/1406.2661)

> [__Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks__](https://arxiv.org/abs/1511.06434)

Following the Original __DCGAN__ Paper Convention :
- they don't use `MaxPooling`

- & the activation function is `LeaklyRelu`

- Here we are Using the `strides=2` to reduce the dimension of Image from MNIST dataset insted of pooling.

- Activation of the Output Dense Layer is `sigmoid` since we are just loking at just binary Classification $\Rightarrow$ __Discriminater__ Only need to tell wether the image is fake or real.

- $64$ $\Rightarrow$ $128$  $\Rightarrow$  $ 256$ as Deeper layer has more filter 



In [ ]:
size = 28
noise_dim = 1

discriminator = Sequential([
    Conv2D(64, 3, strides=2, input_shape=(28, 28, 1)),
    LeakyReLU(),
    BatchNormalization(),
    
    Conv2D(128, 5, strides=2),
    LeakyReLU(),
    BatchNormalization(),
    
    Conv2D(256, 5, strides=2),
    LeakyReLU(),
    BatchNormalization(),
    
    Flatten(),
    Dense(1, activation='sigmoid')
])

opt = tf.keras.optimizers.Adam(lr=2e-4, beta_1=0.5)

discriminator.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 13, 13, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 128)         204928    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 5, 5, 128)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 128)         512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 256)         8

# __4. Generator__

In [ ]:
generator = Sequential([
    Dense(256, activation='relu', input_shape=(noise_dim,)),
    Reshape((1, 1, 256)),
    
    Conv2DTranspose(256, 5, activation='relu'),
    BatchNormalization(),

    Conv2DTranspose(128, 5, activation='relu'),
    BatchNormalization(),

    Conv2DTranspose(64, 5, strides=2, activation='relu'),
    BatchNormalization(),
    
    Conv2DTranspose(32, 5, activation='relu'),
    BatchNormalization(),

    Conv2DTranspose(1, 4, activation='sigmoid')

])

generator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 256)               512       
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 1, 256)         0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 5, 5, 256)         1638656   
_________________________________________________________________
batch_normalization_7 (Batch (None, 5, 5, 256)         1024      
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 9, 9, 128)         819328    
_________________________________________________________________
batch_normalization_8 (Batch (None, 9, 9, 128)         512       
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 21, 21, 64)       

Last layer is `Conv2DTranspose(1, 4, activation='sigmoid')`

- we are using the single filter because that will be our channel information.

- Kernal size = 4, as we are already at 25 by 25 $\rightarrow$ and we need to go atb 28 by 28.

- we take an Input of just one value and genrate the 28 by 28 tensor out of that, which will serve as our genrated image.



In [ ]:
# Generating the random output
noise = np.random.randn(1, noise_dim)
gen_image = generator.predict(noise)[0]

plt.figure()
plt.imshow(np.reshape(gen_image, (28, 28)), cmap='binary')
plt.show()

<IPython.core.display.Javascript object>

# __5. Generative Adversarial Network (GAN)__

In [ ]:
input_layer = tf.keras.layers.Input(shape=(noise_dim,))
gen_out = generator(input_layer)
disc_out = discriminator(gen_out)

gan = Model(
    input_layer,
    disc_out
)

discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 28, 28, 1)         2717025   
_________________________________________________________________
sequential (Sequential)      (None, 1)                 1027073   
Total params: 3,744,098
Trainable params: 2,716,065
Non-trainable params: 1,028,033
_________________________________________________________________


`discriminator.trainable = False` because while the $Generator$ is Training we don't want the $Discriminater$ to be Trained

# __6. Training the GAN__

- We will be create some randomely genrerated examples from  the generater, we also take some true examples from the actul dataset, then these examples are clubbed together with randomely genrated samples to Create a Training batch for __Model__.

- We are dividing the batch size by 2 because we want only half examples which are true, other half will be generated.

- `true_examples` are reshape because we want to add the additional dimension here which dose not exist in the original dataset




In [ ]:
%%time

epochs = 25
batch_size = 128
steps_per_epoch = int(2 * x.shape[0]/batch_size)

print('Steps per epoch=', steps_per_epoch)

dp = tfutils.plotting.DynamicPlot(plt, 5, 5, (8, 8))

for e in range(0, epochs):
    
    dp.start_of_epoch(e)
    
    for step in range(0, steps_per_epoch):
        true_examples = x[int(batch_size/2)*step: int(batch_size/2)*(step + 1)]
        true_examples = np.reshape(true_examples, (true_examples.shape[0], 28, 28, 1))

        noise = np.random.randn(int(batch_size/2), noise_dim)
        generated_examples = generator.predict(noise)

        x_batch = np.concatenate([generated_examples, true_examples], axis=0)
        y_batch = np.array([0] * int(batch_size/2) + [1] * int(batch_size/2))

        indices = np.random.choice(range(batch_size), batch_size, replace=False)
        x_batch = x_batch[indices]
        y_batch = y_batch[indices]

        # train the discriminator
        discriminator.trainable = True
        discriminator.train_on_batch(x_batch, y_batch)
        discriminator.trainable = False

        # train the generator
        loss, _ = gan.train_on_batch(noise, np.ones((int(batch_size/2), 1)))

        _, acc = discriminator.evaluate(x_batch, y_batch, verbose=False)

    noise = np.random.randn(1, noise_dim)
    generated_example = generator.predict(noise)[0]
    
    dp.end_of_epoch(np.reshape(generated_example, (28, 28)), 'binary',
                   'DiscAcc:{:.2f}'.format(acc), 'GANLoss:{:.2f}'.format(loss))

Steps per epoch= 107


<IPython.core.display.Javascript object>

CPU times: user 3min 2s, sys: 12.4 s, total: 3min 14s
Wall time: 3min 23s
